In [21]:
import pandas as pd
import random
import string

In [22]:
def generate_alphabet_tuples():
    return [(chr(i), []) for i in range(ord('a'), ord('z')+1)]


In [23]:
def generate_character_mapping():
    alphabet_tuples = generate_alphabet_tuples()
    for i in range(8):
        ascii_low = string.ascii_lowercase
        for t in alphabet_tuples:
            c =  random.choice(ascii_low)
            t[1].append(c)
            ascii_low =ascii_low.replace(c,'')
    
    
    return alphabet_tuples

In [24]:
alphabet_tuples = generate_character_mapping()

In [25]:
alphabet_tuples

[('a', ['c', 'g', 'q', 'o', 't', 'j', 'l', 's']),
 ('b', ['p', 's', 'm', 'h', 'r', 'a', 'g', 'i']),
 ('c', ['x', 'h', 'z', 'a', 'l', 'r', 'y', 'h']),
 ('d', ['f', 'k', 'h', 'm', 'm', 'd', 'f', 'q']),
 ('e', ['n', 'a', 'i', 'j', 'x', 'c', 'k', 'b']),
 ('f', ['r', 'b', 'd', 'i', 'q', 'p', 'e', 'z']),
 ('g', ['w', 'q', 'o', 'n', 'w', 'l', 'p', 'u']),
 ('h', ['o', 'x', 'f', 'x', 'i', 'x', 'v', 'g']),
 ('i', ['y', 'v', 'y', 'u', 'p', 'o', 'c', 'o']),
 ('j', ['k', 'j', 'p', 'l', 'v', 'w', 'd', 'w']),
 ('k', ['b', 'e', 's', 'v', 's', 's', 'a', 'r']),
 ('l', ['t', 'i', 'g', 'y', 'o', 'b', 'j', 't']),
 ('m', ['e', 'w', 'v', 'c', 'h', 'h', 'w', 'd']),
 ('n', ['v', 'r', 'a', 'p', 'c', 'i', 't', 'n']),
 ('o', ['g', 't', 'l', 'd', 'g', 'q', 'n', 'k']),
 ('p', ['q', 'c', 'k', 'f', 'e', 'g', 'r', 'm']),
 ('q', ['s', 'l', 'w', 't', 'd', 'm', 'm', 'j']),
 ('r', ['j', 'y', 'b', 'z', 'j', 'u', 'x', 'p']),
 ('s', ['a', 'm', 'x', 'b', 'u', 'y', 'q', 'c']),
 ('t', ['u', 'd', 'e', 'w', 'a', 'f', 'u', 'e']),


## change to dataframe 

In [26]:
def generate_alphabet_df():
    # dataframe with indexes from 'a' to 'z' and columns 'value_1' to 'value_8'
    alphabet_df = pd.DataFrame(index=[chr(i) for i in range(ord('a'), ord('d')+1)], 
                               columns=['column_'+str(i) for i in range(1,3)])
    alphabet_df.index.name = "carrier_letter"
    
    
    for each_column in alphabet_df.columns:
        ascii_low = string.ascii_lowercase
        for each_index in alphabet_df.index:
            c = random.choice(ascii_low)
            alphabet_df.loc[each_index, each_column] = c
            ascii_low = ascii_low.replace(c,'')
            
    alphabet_df.reset_index(level=0, inplace=True)
            
    return alphabet_df

In [27]:
alphabet_df = generate_alphabet_df()


In [28]:
print(alphabet_df)

  carrier_letter column_1 column_2
0              a        u        j
1              b        l        l
2              c        v        a
3              d        y        n


In [29]:
def shift_single_column(df, column_name, shift_value =1):
    column_to_shift = df[column_name]
    
    lost_values = column_to_shift[-shift_value:]
    
    shifted_column = df[column_name].shift(shift_value)
   
    df[column_name] = pd.concat([lost_values, shifted_column[shift_value:]]).reset_index(drop=True)

    return df


In [30]:
alphabet_df_shifted = shift_single_column(alphabet_df, 'column_1')

In [31]:
alphabet_df_shifted

,carrier_letter,column_1,column_2
0,a,y,j
1,b,u,l
2,c,l,a
3,d,v,n


## shift columns to achieve no repetition in rows

In [42]:
data = {
    'carrier_letter': ['a', 'b', 'c', 'd'],
    'column_1': ['i', 'z', 'l', 'v'],
    'column_2': ['i', 'l', 'w', 'r'],
    'column_3': ['g', 'h', 'o', 'y'],
    'column_4': ['g', 'i', 'k', 'o'],
    'column_5': ['a', 'b', 'c', 'd'],
}

df = pd.DataFrame(data)

In [33]:
df

,carrier_letter,column_1,column_2
0,a,i,i
1,b,z,l
2,c,l,w
3,d,v,r


In [34]:
df.columns[1:]
# get index of the column by name
column_index = df.columns.get_loc('column_1')
column_index

1

In [35]:
# one random column
column_name = random.choice(df.columns[1:])
column_name

'column_1'

In [ ]:
def move_columns(df,column, column_to_change):
    """Function shifts the values of a column in one column, so there will be no same characters in the same row.
       Function should return df with columns with no same characters in the same row.
    """
    iter = 0
    was_shifted = False
    
    while any(df[column] == df[column_to_change]):
        df = shift_single_column(df, column_to_change)
        was_shifted = True
        iter += 1
        if iter > 100:
            break
    
    return df, was_shifted

In [ ]:
def fix_repetition(df):
    """Function iterates over all columns comparing them and shifting the values; after shift it begins comparing columns from the start"""
    iter = 6
    # skip carrier_letter and first column and fix repetition in the rest of the columns
    for column_to_change in df.columns[2]:
        #comapre
        for column in df.columns[1:iter]:
            if column == column_to_change: # not necessary to check?
                continue
            df , was_shifted = move_columns(df,column, column_to_change)
            
            
        iter -= 1
        
        
        #
        # for column_to_change in df.columns[2:]:
        #     #if we have same column, skip it
        #     if column_to_change == column:
        #         continue
        #     #iterate over all rows and check if there are any same values in the same row
        #     for index, row in df[column].items():

                
        #         if row == df[column_to_change][index]:
        #             #if there are same values in the same row, shift the column
        #             df, was_shifted = move_columns(df,column, column_to_change)
                    
               
            #compare only with columns before the current column
        # for column_to_change in df.columns[2:]:
            #if we have same column, skip it
            #3.compare two values in the same row
            #4.if they are the same, shift the column and check again
            #5. if they are not the same, go to the next row
            #6. if all rows are checked, go to the next column
            
    
        
   
    #7. if all columns are checked, return the dataframe
    return df

In [54]:
df_fixed = fix_repetition(df)


ValueError: too many values to unpack (expected 2)

In [48]:
for column in df.columns[2:]:
    print(df[column])

0    i
1    l
2    w
3    r
Name: column_2, dtype: object
0    g
1    h
2    o
3    y
Name: column_3, dtype: object
0    g
1    i
2    k
3    o
Name: column_4, dtype: object
0    a
1    b
2    c
3    d
Name: column_5, dtype: object
